In [ ]:
import mfem.ser as mfem
from mfem.common.arg_parser import ArgParser
import numpy as np
from glvis import glvis
import matplotlib.pyplot as plt
import mpi4py.MPI as MPI
import h5py


import math
import os
#print(MPI.Get_version())

In [ ]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """
    Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """
    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0 (they will be updated)
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()
    
    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    #mesh.Print("output.mesh")

    return mesh

def gauss_legendre_dirs(N_dir):
    """Compute Gauss-Legendre quadrature points and weights for discrete ordinates.

    This function computes the Gauss-Legendre quadrature points (mu_i) and weights (w_i)
    for the specified number of discrete ordinates using NumPy's polynomial module.

    Args:
        N_dir (int): The number of discrete ordinates.

    Returns:
        tuple: A tuple containing two lists:
            - mu (list of float): The Gauss-Legendre quadrature points.
            - w (list of float): The corresponding Gauss-Legendre quadrature weights.
    """
    gg_mu, gg_w = np.polynomial.legendre.leggauss(N_dir)
    
    mu = gg_mu.tolist()
    w  = gg_w.tolist()
    
    return mu, w

def read_data(nE):
    """Read cross-section and stopping power data from an HDF5 file.

    This function reads data from the file "data.h5". It expects the following datasets:
      - "E_{nE}": Energy array.
      - "E_{nE}": Energy grid array.
      - "xs_t_{nE}": Total cross-section array.
      - "xs_s_{nE}": Scattering cross-section array.
      - "S_{nE}": Stopping power array.

    Args:
        nE (int): Number of energy groups.

    Returns:
        tuple: A tuple containing:
            - E_arr (numpy.ndarray): Energy array.
            - E_grid_arr (numpy.ndarray): Energy grid array.
            - xs_t_arr (numpy.ndarray): Total cross-section array.
            - xs_s_arr (numpy.ndarray): Scattering cross-section array.
            - S_arr (numpy.ndarray): Stopping power array.
    """
    data_file = "data.h5"

    with h5py.File(data_file, "r") as f:
        E_arr = f[f"E_{nE}"][:]
        E_grid_arr = f[f"E_{nE}"][:]
        xs_t_arr = f[f"xs_t_{nE}"][:]
        xs_s_arr = f[f"xs_s_{nE}"][:]
        S_arr    = f[f"S_{nE}"][:]
        
    return E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr

class TotalXSCoefficient(mfem.PyCoefficient):
    """Coefficient for the total cross-section Σ_t(E).

    This coefficient maps the normalized energy coordinate (x[1]) in [0, 1] to the
    corresponding cross-section value. Specifically, a value of y = 0 corresponds to
    E = E_start and y = 1 corresponds to E = E_end.

    Attributes:
        xs_t_data (list or array-like): Total cross-section values for each energy group.
        E_start (float): The starting energy value.
        E_end (float): The ending energy value.
    """

    def __init__(self, xs_t_data, E_start, E_end):
        """Initializes the TotalXSCoefficient instance.

        Args:
            xs_t_data (list or array-like): Array containing the total cross-section values.
            E_start (float): Energy value corresponding to the start of the domain (y = 0).
            E_end (float): Energy value corresponding to the end of the domain (y = 1).
        """
        super(TotalXSCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        """Evaluates the total cross-section at a given coordinate.

        This method converts the normalized energy coordinate found in x[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        total cross-section value.

        Args:
            x (list or array-like): A coordinate array where x[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The total cross-section value corresponding to the computed energy.
        """
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])

'''class TotalXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    cross-section value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(TotalXSCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1] 
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])
'''

class ScatteringXSCoefficient(mfem.PyCoefficient):
    """Coefficient for the scattering cross-section Σ_s(E).

    This coefficient maps the normalized energy coordinate (x[1]) in [0, 1] to the 
    corresponding scattering cross-section value. A value of y = 0 corresponds to E = E_start,
    and y = 1 corresponds to E = E_end.

    Attributes:
        xs_s_data (list or array-like): Array containing the scattering cross-section values.
        E_start (float): The starting energy value corresponding to y = 0.
        E_end (float): The ending energy value corresponding to y = 1.
    """

    def __init__(self, xs_s_data, E_start, E_end):
        """Initializes the ScatteringXSCoefficient instance.

        Args:
            xs_s_data (list or array-like): Array of scattering cross-section values.
            E_start (float): The starting energy value.
            E_end (float): The ending energy value.
        """
        super(ScatteringXSCoefficient, self).__init__()
        self.xs_s_data = xs_s_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        """Evaluates the scattering cross-section at a given normalized energy coordinate.

        This method converts the normalized energy coordinate from x[1] to an energy value
        and maps that to a corresponding energy group, returning the scattering cross-section 
        value from the xs_s_data array.

        Args:
            x (list or array-like): A coordinate array where x[1] is the normalized energy 
                in the range [0, 1].

        Returns:
            float: The scattering cross-section value for the computed energy.
        """
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])
 
'''class ScatteringXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the scattering cross-section Σ_s(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    scattering cross-section value. Here, y=0 corresponds to E = E_start and y=1 
    corresponds to E = E_end.
    """
    def __init__(self, xs_s_data, E_start, E_end):
        super(ScatteringXSCoefficient, self).__init__()
        self.xs_s_data = xs_s_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])
        '''

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """Coefficient for the stopping power S(E).

    This coefficient maps the normalized energy coordinate (x[1]) in the interval [0, 1]
    to the corresponding stopping power S(E). Specifically, a value of y = 0 corresponds to 
    E = E_start, and a value of y = 1 corresponds to E = E_end.

    Attributes:
        S_data (list or array-like): Array of stopping power values for each energy group.
        E_start (float): The starting energy value (corresponding to y = 0).
        E_end (float): The ending energy value (corresponding to y = 1).
    """

    def __init__(self, S_data, E_start, E_end):
        """Initializes the StoppingPowerCoefficient instance.

        Args:
            S_data (list or array-like): Array containing the stopping power values.
            E_start (float): The starting energy value.
            E_end (float): The ending energy value.
        """
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        """Evaluates the stopping power at a given normalized energy coordinate.

        Converts the normalized energy coordinate in x[1] to the actual energy value
        and retrieves the corresponding stopping power from S_data.

        Args:
            x (list or array-like): A coordinate array where x[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The stopping power value corresponding to the computed energy.
        """
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])

'''class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    S(E) value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])
'''
    
class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    This coefficient maps the normalized energy coordinate (x[1]) in the interval [0, 1]
    to the corresponding derivative of the stopping power S(E). A value of y = 0 corresponds 
    to E = E_start and y = 1 corresponds to E = E_end.

    Attributes:
        dS_data (list or array-like): Array containing the derivative values of the stopping power for each energy group.
        E_start (float): The starting energy value corresponding to y = 0.
        E_end (float): The ending energy value corresponding to y = 1.
    """

    def __init__(self, dS_data, E_start, E_end):
        """Initializes the StoppingPowerDerivativeCoefficient instance.

        Args:
            dS_data (list or array-like): Array containing the derivative values of the stopping power.
            E_start (float): The starting energy value.
            E_end (float): The ending energy value.
        """
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        self.dS_data = dS_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        """Evaluates the derivative of the stopping power at a given normalized energy coordinate.

        This method converts the normalized energy coordinate in x[1] to the corresponding energy
        value and determines the appropriate energy group. It then returns the derivative value from
        the dS_data array corresponding to that energy group.

        Args:
            x (list or array-like): A coordinate array where x[1] is the normalized energy (in the range [0, 1]).

        Returns:
            float: The derivative of the stopping power corresponding to the computed energy.
        """
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])

'''class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    derivative value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, dS_data, E_start, E_end):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        self.dS_data = dS_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])
'''

class Inflow2DCoefficient(mfem.PyCoefficient):
    """Coefficient for the inflow boundary condition.

    Returns a prescribed inflow flux value.

    Attributes:
        inflow_value (float): The constant inflow flux value to be used.
    """

    def __init__(self, inflow_value):
        """Initializes the Inflow2DCoefficient with a given inflow flux value.

        Args:
            inflow_value (float): The inflow flux value that will be returned by EvalValue.
        """
        super(Inflow2DCoefficient, self).__init__()
        self.inflow_value = inflow_value

    def EvalValue(self, x):
        """Evaluates the inflow coefficient at a given coordinate.

        This method returns the prescribed constant inflow flux value, irrespective of the 
        input spatial coordinate.

        Args:
            x (iterable): The spatial coordinate (ignored in this coefficient).

        Returns:
            float: The prescribed inflow flux value.
        """
        return self.inflow_value

'''class Inflow2DCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the inflow boundary condition.
    Returns a prescribed inflow flux value.
    """
    def __init__(self, inflow_value):
        super(Inflow2DCoefficient, self).__init__()
        self.inflow_value = inflow_value

    def EvalValue(self, x):
        return self.inflow_value'''
       
class QCoefficient(mfem.PyCoefficient):
    """Coefficient for the source term Q(x, E).

    This coefficient maps the normalized energy coordinate (x[1] in [0, 1]) to the corresponding
    source term Q value. Here, y = 0 corresponds to E = E_start and y = 1 corresponds to E = E_end.

    If Q_data is given as the scalar 0, then the coefficient always returns 0 and E_start and E_end are not required.
    """

    def __init__(self, Q_data, E_start=None, E_end=None):
        """Initializes the QCoefficient.

        Args:
            Q_data (int, float, or list): The source term data. If a scalar equal to 0 is provided, the
                coefficient will always return 0. Otherwise, if a nonzero scalar is provided, it is converted
                to a single-element list.
            E_start (float, optional): The starting energy value. Must be provided if Q_data is non-zero.
            E_end (float, optional): The ending energy value. Must be provided if Q_data is non-zero.

        Raises:
            ValueError: If non-zero Q_data is provided without both E_start and E_end.
        """
        super(QCoefficient, self).__init__()

        if isinstance(Q_data, (int, float)) and Q_data == 0:
            self.scalar_zero = True
            self.Q_data = None
        else:
            if E_start is None or E_end is None:
                raise ValueError("For non-zero Q_data, E_start and E_end must be provided.")
            self.scalar_zero = False
            if isinstance(Q_data, (int, float)):
                self.Q_data = [Q_data]
            else:
                self.Q_data = Q_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, x):
        """Evaluates the source term Q at a given coordinate.

        For a non-zero Q_data, this method converts the normalized energy coordinate x[1] to the corresponding
        energy value and returns the Q value for the associated energy group. If Q_data is zero, it always returns 0.

        Args:
            x (list or array-like): The coordinate array where x[1] is the normalized energy (in [0, 1]).

        Returns:
            float: The source term Q value corresponding to the computed energy, or 0.0 if Q_data is zero.
        """
        if self.scalar_zero:
            return 0.0

        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.Q_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.Q_data[group])

'''class QCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the source term Q(x,E).

    Maps the normalized energy coordinate (x[1] in [0,1]) to the corresponding 
    Q value. Here, y=0 corresponds to E = E_start and y=1 corresponds to E = E_end.
    
    If Q_data is given as the scalar 0, then the coefficient always returns 0,
    and E_start and E_end are not required.
    """
    def __init__(self, Q_data, E_start=None, E_end=None):
        super(QCoefficient, self).__init__()

        if isinstance(Q_data, (int, float)) and Q_data == 0:
            self.scalar_zero = True
            self.Q_data = None

        else:
            if E_start is None or E_end is None:
                raise ValueError("For non-zero Q_data, E_start and E_end must be provided.")
            self.scalar_zero = False
            if isinstance(Q_data, (int, float)):

                self.Q_data = [Q_data]
            else:
                self.Q_data = Q_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, x):
        if self.scalar_zero:
            return 0.0
        
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.Q_data)
        
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))

        return float(self.Q_data[group])
'''
    
def compute_S_derivative(E, S):
    """Compute the derivative dS/dE using finite differences.

    This function approximates the derivative of S with respect to E by applying finite differences.
    It uses a forward difference at the first point, a backward difference at the last point, and
    central differences for all interior points.

    Args:
        E (numpy.ndarray): 1D array of energy values.
        S (numpy.ndarray): 1D array of S values corresponding to the energy values.

    Returns:
        numpy.ndarray: Array of the finite difference approximations of dS/dE.
    """
    n = len(S)
    dS_dE = np.zeros(n)
    
    # Forward difference for the first point
    dS_dE[0] = (S[1] - S[0]) / (E[1] - E[0])
    
    # Central difference for interior points
    for i in range(1, n - 1):
        dS_dE[i] = (S[i + 1] - S[i - 1]) / (E[i + 1] - E[i - 1])
    
    # Backward difference for the last point
    dS_dE[-1] = (S[-1] - S[-2]) / (E[-1] - E[-2])
    
    return dS_dE

'''def compute_S_derivative(E, S):

    """
    Compute the derivative dS/dE using finite differences.
    
    Args:
        E (ndarray): 1D array of energy values.
        S (ndarray): 1D array of S values corresponding to the energies.
    
    Returns:
        dS_dE (ndarray): Array of the finite difference approximation of the derivative dS/dE.
    """
    n = len(S)
    dS_dE = np.zeros(n)
    
    # Forward difference for the first point
    dS_dE[0] = (S[1] - S[0]) / (E[1] - E[0])
    
    # Central difference for interior points
    for i in range(1, n - 1):
        dS_dE[i] = (S[i + 1] - S[i - 1]) / (E[i + 1] - E[i - 1])
    
    # Backward difference for the last point
    dS_dE[-1] = (S[-1] - S[-2]) / (E[-1] - E[-2])
    
    return dS_dE'''

'''class MeshTransformer2D:
    """
    Transforms a 2D mesh defined on the unit square [0,1]^2 into the domain 
    [x_start, x_end] x [E_start, E_end]. The first coordinate (x) is scaled by L, and the 
    second coordinate (interpreted as a normalized energy variable) is mapped 
    linearly from [0,1] to [E_start, E_end].
    """
    def __init__(self, x_start, x_end, E_start, E_end):
        self.L = x_end - x_start
        self.E_start = E_start
        self.E_end = E_end

    def Transform(self, mesh):
        """
        Transforms the vertices of the given mesh.

        Parameters:
          mesh : an mfem.Mesh instance defined on [0,1]^2.
        """
        # Get the vertex array (returned as a tuple) and convert it to a NumPy array.
        verts = np.array(mesh.GetVertexArray())
        
        # Assuming the mesh is 2D:
        # - Multiply the x-coordinate (column 0) by L.
        # - Map the second coordinate (energy) from [0,1] to [E_start, E_end].
        verts[:, 0] *= self.L
        verts[:, 1] = self.E_start + verts[:, 1] * (self.E_end - self.E_start)
        
        # Flatten the 2D vertex array into a 1D list (row-major order).
        flat_verts = verts.flatten().tolist()
        
        # Create an mfem.Vector from the flattened list.
        v_vec = mfem.Vector(flat_verts)
        
        # Update the mesh vertices.
        mesh.SetVertices(v_vec)
        return mesh

class VelocityCoefficient(mfem.VectorPyCoefficientBase):
    """
    Coefficient for the velocity vector in the transport equation.
    
    This coefficient defines:
         v(x) = [ μ, S(E) ]
    where:
       - μ is a provided scalar (e.g., the discrete ordinate value),
       - S(E) is computed by S_coef.Eval(x) using the energy coordinate x[1].
    
    The class inherits from VectorPyCoefficientBase (the Python wrapper for the
    C++ VectorPyCoefficientBase) and overrides the _EvalPy method.
    """
    def __init__(self, mu, S_coef):
        mfem.VectorPyCoefficientBase.__init__(self, 2, 0)
        self.mu = mu
        self.S_coef = S_coef
    def _EvalPy(self, V, ip):
        V[0] = self.mu
        V[1] = self.S_coef.EvalValue(ip)
'''


'class MeshTransformer2D:\n    """\n    Transforms a 2D mesh defined on the unit square [0,1]^2 into the domain \n    [x_start, x_end] x [E_start, E_end]. The first coordinate (x) is scaled by L, and the \n    second coordinate (interpreted as a normalized energy variable) is mapped \n    linearly from [0,1] to [E_start, E_end].\n    """\n    def __init__(self, x_start, x_end, E_start, E_end):\n        self.L = x_end - x_start\n        self.E_start = E_start\n        self.E_end = E_end\n\n    def Transform(self, mesh):\n        """\n        Transforms the vertices of the given mesh.\n\n        Parameters:\n          mesh : an mfem.Mesh instance defined on [0,1]^2.\n        """\n        # Get the vertex array (returned as a tuple) and convert it to a NumPy array.\n        verts = np.array(mesh.GetVertexArray())\n        \n        # Assuming the mesh is 2D:\n        # - Multiply the x-coordinate (column 0) by L.\n        # - Map the second coordinate (energy) from [0,1] to [E_start, E_en

In [1]:
# This will be the init function:
nx = 20
nE = 50
x_start = 0.0
x_end = 0.3
E_start = 1.0
E_end = 0.01
N_ang = 8
order = 1


#mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
#dim = mesh.Dimension()

#fec = mfem.DG_FECollection(order, dim)
#fes = mfem.FiniteElementSpace(mesh, fec)

#ndofs = fes.GetVSize()
#print(f"Number of dimensions: {dim}")
#print(f"Number of finite element unknowns: {ndofs}")

#a = mfem.BilinearForm(fes)

#ess_tdof_list = mfem.intArray()
#dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
#dir_bdr.Assign(1)
#dir_bdr.Size()

#print(f"Number of finite element dir_bdr.Size(): {dir_bdr.Size()}")
#print(f"Number of finite element ess_tdof_list.Size(): {ess_tdof_list.Size()}")

#inflow_coeff = Inflow2DCoefficient(1.0)

# Read cross-section and stopping power data and compute necessary coefficients
E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
dS_dE_arr = compute_S_derivative(E_arr, -S_arr)
xs_t_coeff = TotalXSCoefficient(xs_t_arr, E_start, E_end)
xs_s_coeff = ScatteringXSCoefficient(xs_s_arr, E_start, E_end)
S_coeff = StoppingPowerCoefficient(-S_arr, E_start, E_end)
dS_dE_coeff = StoppingPowerDerivativeCoefficient(dS_dE_arr, E_start, E_end)

# Get Gauss-Legendre angular quadrature points and weights
mu_vals, w_vals = gauss_legendre_dirs(N_ang)

# Create a 2D mesh and finite element space (this remains fixed for all iterations)
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
#transformer = MeshTransformer2D(x_start, x_end, E_start, E_end)
#transformer.Transform(mesh)
dim = mesh.Dimension()
fec = mfem.L2_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
print("Number of unknowns:", fes.GetTrueVSize())

# Initialize the total solution phi (scattering source) to 1 everywhere.
phi = mfem.GridFunction(fes)
phi.Assign(1.0)

# Source iteration parameters
max_iter = 20
tol = 1e-6

# Start the source iteration loop
for iter in range(max_iter):
    print("Source iteration, iter =", iter)
    psi_mu_list = []  # To store tuples (mu, weight, psi) for each angular direction

    # Angular sweep: solve for each angular direction μ
    for mu, w in zip(mu_vals, w_vals):
        print("  Solving for μ =", mu)
        # Define the velocity coefficient v_coef depending on μ and S(E)
        v_coef = VelocityCoefficient(mu, S_coeff)

        # Set up the DG bilinear form a(.,.) on the fixed FE space.
        # The integrators include:
        # - A ConvectionIntegrator for the term: μ∂ψ/∂x + S(E)∂ψ/∂E
        # - Two MassIntegrators for the reaction term: [S'(E) + Σₜ(E)] ψ and for material interaction
        # - Face integrators for interior and boundary faces via NonconservativeDGTraceIntegrator.
        a = mfem.BilinearForm(fes)
        a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coef))
        a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
        a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
        a.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
        a.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
        a.Assemble()
        a.Finalize()

        # Set up the linear form b(·) for the right-hand side.
        # Here, we add the boundary inflow condition and a scattering source term.
        b = mfem.LinearForm(fes)
        inflow_2d = Inflow2DCoefficient(1.0)
        b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_2d, v_coef, 1.0))
        # Add scattering source: xs_s_coeff * φ (current source iterate)
        #scattering_source = mfem.ProductCoefficient(xs_s_coeff, mfem.GridFunctionCoefficient(phi))
        #b.AddDomainIntegrator(mfem.DomainLFIntegrator(scattering_source))
        b.Assemble()

        # Solve the linear system Aψ = b using GMRES.
        A = a.SpMat()
        prec = mfem.GSSmoother(A)
        solver = mfem.GMRESSolver()
        solver.SetOperator(A)
        solver.SetPreconditioner(prec)
        solver.SetRelTol(1e-12)
        solver.SetAbsTol(1e-12)
        solver.SetMaxIter(500)
        solver.SetKDim(30)
        solver.SetPrintLevel(0)

        # Initialize the angular solution ψ for this μ.
        psi = mfem.GridFunction(fes)
        psi.Assign(1.0)
        solver.Mult(b, psi)
        print("    GMRES iterations (μ = {}):".format(mu), solver.GetNumIterations(), " final norm:", solver.GetFinalNorm())
        psi_mu_list.append((mu, w, psi))

    # Angular integration: compute the updated total solution φ_new(x,E) = Σ_μ w(μ) ψ(x,E,μ)
    phi_new = mfem.GridFunction(fes)
    phi_new.Assign(0.0)
    for mu, w, psi in psi_mu_list:
        phi_new.Add(w, psi)

    # Check convergence of the source iteration by computing the L2 norm of (phi_new - phi)
    diff_gf = mfem.GridFunction(fes)
    diff_gf.Assign(phi_new)
    diff_gf.Add(-1.0, phi)
    diff = diff_gf.Norml2()

    print("  Source iteration diff =", diff)
    if diff < tol:
        phi.Assign(phi_new)
        print("Source iteration converged.")
        break
    phi.Assign(phi_new)                





'''
for mu, w in zip(mu_vals, w_vals):
    print("Solving for μ =", mu)
    # Create a 2D mesh over [0,L] x [E_min,E_max] using quadrilaterals.
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
    #transformer = MeshTransformer2D(x_start, x_end, E_start, E_end)
    #transformer.Transform(mesh)
    dim = mesh.Dimension()
    order = 2

    # Define the DG finite element space using L2 elements
    fec = mfem.L2_FECollection(order, dim)
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetTrueVSize())
    v_coef = VelocityCoefficient(mu, S_coeff)

    # Set up the bilinear form a(.,.) for the DG formulation.
    a = mfem.BilinearForm(fes)
    # Volume integrator for convection term: μ ∂ψ/∂x + S(E) ∂ψ/∂E.
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coef))
    # Mass integrators for the reaction term: [S'(E) + Σₜ(E)] ψ.
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
    a.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
    a.Assemble()
    a.Finalize()
    
    b = mfem.LinearForm(fes)
    inflow_2d = Inflow2DCoefficient(1.0)
    q_coef = QCoefficient(0)
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_2d, v_coef, 1.0))
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))
    b.Assemble()

    # Solve the linear system using GMRES
    A = a.SpMat()
    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(500)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)

    # Initialize the solution GridFunction
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)
    solver.Mult(b, psi)
    print("GMRES iterations (μ = {}):".format(mu), solver.GetNumIterations(), " final norm:", solver.GetFinalNorm())
    # Save the solution for this angular direction and store in list
    psi_mu_list.append((mu, psi))
    out_name = f"psi_mu_{mu:.3f}.gf"
    psi.Save(out_name)
    print(f"Solution for μ = {mu:.3f} saved to {out_name}.\n")

    
# Angular integration: compute total solution phi(x,E) = Σ_{μ} w(μ) ψ(x,E,μ)

phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, psi in psi_mu_list:
    weight = w_vals[mu_vals.index(mu)]
    phi.Add(weight, psi)
out_total = "phi.gf"
phi.Save(out_total)
print("Angle-integrated solution saved to", out_total)
# Save the mesh for GLVis visualization
mesh.Print("mesh.mesh")
print("Mesh saved to mesh.mesh")


#---------------------------------------------------
# GLVis Visualization
#---------------------------------------------------
# Visualize the total (angle-integrated) solution with GLVis.
# The title is set to "Total Angular Flux" and we wait for 500 ms.
glvis(mesh, phi, "Total Angular Flux", 500)
#---------------------------------------------------
# Dose Calculation and Plotting (Matplotlib)
#---------------------------------------------------
# Here, we compute the average flux in the energy (y) direction for each x cell,
# then define dose as the normalized flux relative to the value at x=0.
x_coords = []
avg_flux = []
for i in range(nx):
    # x coordinate of the cell center
    x_val = x_start + (x_end - x_start)*(i + 0.5)/ nx
    cell_vals = []
    for j in range(ny):
        # y (energy) coordinate of the cell center
        y_val = E_start + (E_end - E_start)*(j + 0.5) / nE
        # Compute cell index in row-major order
        cell_index = i * nE + j
        cell_vals.append(phi[cell_index])
    x_coords.append(x_val)
    avg_flux.append(np.mean(cell_vals))
avg_flux = np.array(avg_flux)
# Define dose as the normalized average flux (dose at x divided by dose at x=0)
dose = avg_flux / avg_flux[0]
# Plot the average flux along x
plt.figure()
plt.plot(x_coords, avg_flux, 'o-', label='Average Flux')
plt.xlabel("x (cm)")
plt.ylabel("Average Angular Flux ψ(x)")
plt.title("Average Angular Flux vs. x")
plt.grid(True)
plt.legend()
# Plot the computed dose profile
plt.figure()
plt.plot(x_coords, dose, 's-', color='red', label='Dose (Normalized)')
plt.xlabel("x (cm)")
plt.ylabel("Dose (normalized)")
plt.title("Dose Profile vs. x")
plt.grid(True)
plt.legend()
plt.show()


'''






NameError: name 'read_data' is not defined

In [107]:
'''b = mfem.LinearForm(fes)
q_coef = QCoefficient(0)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))
b.Assemble()'''

'b = mfem.LinearForm(fes)\nq_coef = QCoefficient(0)\nb.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))\nb.Assemble()'

In [114]:
'''
# This will be the init function:
nx = 20
nE = 50
x_start = 0.0
x_end = 0.3
E_start = 1.0
E_end = 0.01
N_ang = 8
order = 1


#mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
#dim = mesh.Dimension()

#fec = mfem.DG_FECollection(order, dim)
#fes = mfem.FiniteElementSpace(mesh, fec)

#ndofs = fes.GetVSize()
#print(f"Number of dimensions: {dim}")
#print(f"Number of finite element unknowns: {ndofs}")

#a = mfem.BilinearForm(fes)

#ess_tdof_list = mfem.intArray()
#dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
#dir_bdr.Assign(1)
#dir_bdr.Size()

#print(f"Number of finite element dir_bdr.Size(): {dir_bdr.Size()}")
#print(f"Number of finite element ess_tdof_list.Size(): {ess_tdof_list.Size()}")

#inflow_coeff = Inflow2DCoefficient(1.0)

# Read cross-section and stopping power data and compute necessary coefficients
E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
dS_dE_arr = compute_S_derivative(E_arr, -S_arr)
xs_t_coeff = TotalXSCoefficient(xs_t_arr, E_start, E_end)
xs_s_coeff = ScatteringXSCoefficient(xs_s_arr, E_start, E_end)
S_coeff = StoppingPowerCoefficient(-S_arr, E_start, E_end)
dS_dE_coeff = StoppingPowerDerivativeCoefficient(dS_dE_arr, E_start, E_end)

# Get Gauss-Legendre angular quadrature points and weights
mu_vals, w_vals = gauss_legendre_dirs(N_ang)

# Create a 2D mesh and finite element space (this remains fixed for all iterations)
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
#transformer = MeshTransformer2D(x_start, x_end, E_start, E_end)
#transformer.Transform(mesh)
dim = mesh.Dimension()
fec = mfem.L2_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
print("Number of unknowns:", fes.GetTrueVSize())

# Initialize the total solution phi (scattering source) to 1 everywhere.
phi = mfem.GridFunction(fes)
phi.Assign(1.0)

# Source iteration parameters
max_iter = 20
tol = 1e-6

# Start the source iteration loop
for iter in range(max_iter):
    print("Source iteration, iter =", iter)
    psi_mu_list = []  # To store tuples (mu, weight, psi) for each angular direction

    # Angular sweep: solve for each angular direction μ
    for mu, w in zip(mu_vals, w_vals):
        print("  Solving for μ =", mu)
        # Define the velocity coefficient v_coef depending on μ and S(E)
        v_coef = VelocityCoefficient(mu, S_coeff)

        # Set up the DG bilinear form a(.,.) on the fixed FE space.
        # The integrators include:
        # - A ConvectionIntegrator for the term: μ∂ψ/∂x + S(E)∂ψ/∂E
        # - Two MassIntegrators for the reaction term: [S'(E) + Σₜ(E)] ψ and for material interaction
        # - Face integrators for interior and boundary faces via NonconservativeDGTraceIntegrator.
        a = mfem.BilinearForm(fes)
        a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coef))
        a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
        a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
        a.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
        a.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
        a.Assemble()
        a.Finalize()

        # Set up the linear form b(·) for the right-hand side.
        # Here, we add the boundary inflow condition and a scattering source term.
        b = mfem.LinearForm(fes)
        inflow_2d = Inflow2DCoefficient(1.0)
        b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_2d, v_coef, 1.0))
        # Add scattering source: xs_s_coeff * φ (current source iterate)
        scattering_source = mfem.ProductCoefficient(xs_s_coeff, mfem.GridFunctionCoefficient(phi))
        b.AddDomainIntegrator(mfem.DomainLFIntegrator(scattering_source))
        b.Assemble()

        # Solve the linear system Aψ = b using GMRES.
        A = a.SpMat()
        prec = mfem.GSSmoother(A)
        solver = mfem.GMRESSolver()
        solver.SetOperator(A)
        solver.SetPreconditioner(prec)
        solver.SetRelTol(1e-12)
        solver.SetAbsTol(1e-12)
        solver.SetMaxIter(500)
        solver.SetKDim(30)
        solver.SetPrintLevel(0)

        # Initialize the angular solution ψ for this μ.
        psi = mfem.GridFunction(fes)
        psi.Assign(1.0)
        solver.Mult(b, psi)
        print("    GMRES iterations (μ = {}):".format(mu), solver.GetNumIterations(), " final norm:", solver.GetFinalNorm())
        psi_mu_list.append((mu, w, psi))

    # Angular integration: compute the updated total solution φ_new(x,E) = Σ_μ w(μ) ψ(x,E,μ)
    phi_new = mfem.GridFunction(fes)
    phi_new.Assign(0.0)
    for mu, w, psi in psi_mu_list:
        phi_new.Add(w, psi)

    # Check convergence of the source iteration by computing the L2 norm of (phi_new - phi)
    diff_gf = mfem.GridFunction(fes)
    diff_gf.Assign(phi_new)
    diff_gf.Add(-1.0, phi)
    diff = diff_gf.Norml2()

    print("  Source iteration diff =", diff)
    if diff < tol:
        phi.Assign(phi_new)
        print("Source iteration converged.")
        break
    phi.Assign(phi_new)                
'''




'''
for mu, w in zip(mu_vals, w_vals):
    print("Solving for μ =", mu)
    # Create a 2D mesh over [0,L] x [E_min,E_max] using quadrilaterals.
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
    #transformer = MeshTransformer2D(x_start, x_end, E_start, E_end)
    #transformer.Transform(mesh)
    dim = mesh.Dimension()
    order = 2

    # Define the DG finite element space using L2 elements
    fec = mfem.L2_FECollection(order, dim)
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetTrueVSize())
    v_coef = VelocityCoefficient(mu, S_coeff)

    # Set up the bilinear form a(.,.) for the DG formulation.
    a = mfem.BilinearForm(fes)
    # Volume integrator for convection term: μ ∂ψ/∂x + S(E) ∂ψ/∂E.
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coef))
    # Mass integrators for the reaction term: [S'(E) + Σₜ(E)] ψ.
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
    a.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, 1.0))
    a.Assemble()
    a.Finalize()
    
    b = mfem.LinearForm(fes)
    inflow_2d = Inflow2DCoefficient(1.0)
    q_coef = QCoefficient(0)
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_2d, v_coef, 1.0))
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))
    b.Assemble()

    # Solve the linear system using GMRES
    A = a.SpMat()
    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(500)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)

    # Initialize the solution GridFunction
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)
    solver.Mult(b, psi)
    print("GMRES iterations (μ = {}):".format(mu), solver.GetNumIterations(), " final norm:", solver.GetFinalNorm())
    # Save the solution for this angular direction and store in list
    psi_mu_list.append((mu, psi))
    out_name = f"psi_mu_{mu:.3f}.gf"
    psi.Save(out_name)
    print(f"Solution for μ = {mu:.3f} saved to {out_name}.\n")

    
# Angular integration: compute total solution phi(x,E) = Σ_{μ} w(μ) ψ(x,E,μ)

phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, psi in psi_mu_list:
    weight = w_vals[mu_vals.index(mu)]
    phi.Add(weight, psi)
out_total = "phi.gf"
phi.Save(out_total)
print("Angle-integrated solution saved to", out_total)
# Save the mesh for GLVis visualization
mesh.Print("mesh.mesh")
print("Mesh saved to mesh.mesh")


#---------------------------------------------------
# GLVis Visualization
#---------------------------------------------------
# Visualize the total (angle-integrated) solution with GLVis.
# The title is set to "Total Angular Flux" and we wait for 500 ms.
glvis(mesh, phi, "Total Angular Flux", 500)
#---------------------------------------------------
# Dose Calculation and Plotting (Matplotlib)
#---------------------------------------------------
# Here, we compute the average flux in the energy (y) direction for each x cell,
# then define dose as the normalized flux relative to the value at x=0.
x_coords = []
avg_flux = []
for i in range(nx):
    # x coordinate of the cell center
    x_val = x_start + (x_end - x_start)*(i + 0.5)/ nx
    cell_vals = []
    for j in range(ny):
        # y (energy) coordinate of the cell center
        y_val = E_start + (E_end - E_start)*(j + 0.5) / nE
        # Compute cell index in row-major order
        cell_index = i * nE + j
        cell_vals.append(phi[cell_index])
    x_coords.append(x_val)
    avg_flux.append(np.mean(cell_vals))
avg_flux = np.array(avg_flux)
# Define dose as the normalized average flux (dose at x divided by dose at x=0)
dose = avg_flux / avg_flux[0]
# Plot the average flux along x
plt.figure()
plt.plot(x_coords, avg_flux, 'o-', label='Average Flux')
plt.xlabel("x (cm)")
plt.ylabel("Average Angular Flux ψ(x)")
plt.title("Average Angular Flux vs. x")
plt.grid(True)
plt.legend()
# Plot the computed dose profile
plt.figure()
plt.plot(x_coords, dose, 's-', color='red', label='Dose (Normalized)')
plt.xlabel("x (cm)")
plt.ylabel("Dose (normalized)")
plt.title("Dose Profile vs. x")
plt.grid(True)
plt.legend()
plt.show()


'''


'\nfor mu, w in zip(mu_vals, w_vals):\n    print("Solving for μ =", mu)\n    # Create a 2D mesh over [0,L] x [E_min,E_max] using quadrilaterals.\n    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)\n    #transformer = MeshTransformer2D(x_start, x_end, E_start, E_end)\n    #transformer.Transform(mesh)\n    dim = mesh.Dimension()\n    order = 2\n\n    # Define the DG finite element space using L2 elements\n    fec = mfem.L2_FECollection(order, dim)\n    fes = mfem.FiniteElementSpace(mesh, fec)\n    print("Number of unknowns:", fes.GetTrueVSize())\n    v_coef = VelocityCoefficient(mu, S_coeff)\n\n    # Set up the bilinear form a(.,.) for the DG formulation.\n    a = mfem.BilinearForm(fes)\n    # Volume integrator for convection term: μ ∂ψ/∂x + S(E) ∂ψ/∂E.\n    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coef))\n    # Mass integrators for the reaction term: [S\'(E) + Σₜ(E)] ψ.\n    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))\n    a.AddDomainIntegrator(mfem.M

In [110]:
def create_3D_mesh(nx, ny, nz, x_start, x_end, y_start, y_end, z_start, z_end):
    """
    Creates a 3D mesh with specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        nz (int): Number of intervals in the z-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.
        z_start (float): Starting z-coordinate.
        z_end (float): Ending z-coordinate.

    Returns:
        mesh: The updated 3D mesh with vertex coordinates set accordingly.
    """
    # Generate equally spaced coordinates for x, y, and z
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    z_coords = np.linspace(z_start, z_end, nz + 1)
    
    # Create the 3D mesh with initial x, y, z values set to 0
    mesh = mfem.Mesh(nx, ny, nz, "HEXAHEDRON", True, 0.0, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Expected number of vertices: (nx+1) * (ny+1) * (nz+1)
    expected_num = (nx + 1) * (ny + 1) * (nz + 1)
    num_verts = mesh.GetNV()
    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in order: x changes fastest, then y, then z.
    k = 0
    for k_z in range(nz + 1):
        for k_y in range(ny + 1):
            for k_x in range(nx + 1):
                verts[k][0] = x_coords[k_x]
                verts[k][1] = y_coords[k_y]
                verts[k][2] = z_coords[k_z]
                k += 1
    
    # Define the target directory and file name
    target_dir = os.path.join(os.getcwd(), 'mesh', 'usr')
    file_name = f'{nx}x{ny}_3D.mesh'
    file_path = os.path.join(target_dir, file_name)
    
    # Check if the target directory exists; if not, create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Directory '{target_dir}' was created.")

    # Check if the file already exists
    if not os.path.exists(file_path):
        # If the file does not exist, write the mesh to the file
        mesh.Print(file_path)
        print(f"File '{file_path}' was successfully created.")
    else:
        print(f"File '{file_path}' already exists.")

    return mesh

In [111]:
glvis(mesh, 900, 900)

_GlvisWidgetCore(data_str='mesh\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POINT   …

In [112]:
mesh1 = create_3D_mesh(20, 20, 50, 0, 1, 0, 1, 1, 0.01)
glvis(mesh1, 900, 900)

File '/Users/melekderman/github/BFP/examples/mesh/usr/20x20_3D.mesh' already exists.


_GlvisWidgetCore(data_str='mesh\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POINT   …